In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
print('hello world!')

In [36]:
#import visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
#import machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

#import machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split #split
from sklearn.metrics import accuracy_score #metrics

#tools for hyperparameters search
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import os
print(os.listdir("../input"))

In [13]:
data = pd.read_csv('../input/bank-marketing-dataset/bank.csv')

In [14]:
data.head()

In [15]:
#check how many rows are in the dataset 
print('There are {rows} rows in the dataset.'.format(rows = len(data)))

In [16]:
#scan the dataset for null or missing values 
missing_values = data.isnull().mean()*100

missing_values.sum()

In [17]:
#clean data and prepare for machine learning model
#our dataset has categorical data so we need to prepare the dataset for processing

def dummy_bool_values(row, column_name):
    return 1 if row[column_name] == 'yes' else 0

def corrected_values(row, column_name, threshold, data):
    if row[column_name] <= threshold:
        return row[column_name]
    else:
        mean = data[data[column_name] <= threshold][column_name].mean()
        return mean

def processed_df(data):
    cleaned_data = data.copy()
    
    boolean_columns = ['default', 'housing', 'loan', 'deposit']
    for bool_columns in boolean_columns:
        cleaned_data[bool_columns + '_bool'] = data.apply(lambda row: dummy_bool_values(row, bool_columns),axis=1)
    
    cleaned_data = cleaned_data.drop(columns = boolean_columns)
    
    #now we can convert categorical columns to dummies
    
    categorical_columns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
    
    for new_col in  categorical_columns:
        cleaned_data = pd.concat([cleaned_data.drop(new_col, axis=1),
                                pd.get_dummies(cleaned_data[new_col], prefix=new_col, prefix_sep='_',
                                               drop_first=True, dummy_na=False)], axis=1)
    
    #impute the incorrect values and drop original columns
    cleaned_data['campaign_cleaned'] = data.apply(lambda row: corrected_values(row, 'campaign', 34, cleaned_data),axis=1)
    cleaned_data['previous_cleaned'] = data.apply(lambda row: corrected_values(row, 'previous', 34, cleaned_data),axis=1)
    
    cleaned_data = cleaned_data.drop(columns = ['campaign', 'previous'])
    
    return cleaned_data

In [18]:
#clean the dataset
cleaned_data = processed_df(data)
cleaned_data.head(20)

In [19]:
help(train_test_split)

In [20]:
#splitting my cleaned data set
X = cleaned_data.drop(columns = 'deposit_bool')
y = cleaned_data[['deposit_bool']]

In [21]:
#setting test size and randome state 
TEST_SIZE = 0.2
RAND_STATE = 42

In [24]:
#split the dataset and print out the shapes for each set to verify 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=RAND_STATE)
display('This is how the dataset is split', X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [25]:
nb = GaussianNB()

In [29]:
y_train = np.ravel(y_train)
nb.fit(X_train,y_train)

In [30]:
y_res = nb.predict(X_test)

In [33]:
print(y_res)

In [34]:
print(y_test)

In [39]:
r = sklearn.metrics.confusion_matrix(y_test, y_res)
accuracy = (r[0][0] + r[-1][-1]) / np.sum(r)
print(accuracy)